# Train Your Own Reasoning Model in 48 Hours on a Single GPU

This tutorial provides a tried-and-true recipe for training your own reasoning model by fine-tuning a [Meta LLaMA 3.1–8B Instruct](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct) model with NVIDIA NeMo in about 48 hours on a single H100 80GB GPU.

This recipe is inspired by the [Llama Nemotron family of models](https://www.nvidia.com/en-us/ai-data-science/foundation-models/llama-nemotron/), where the model can selectively turn reasoning on or off based on instructions in the system prompt. You'll train your model on complex instruction-following and reasoning tasks using the [Llama-Nemotron-Post-Training-Data](https://huggingface.co/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset).

### ✅ What You'll Learn
1. An effective recipe to train your own reasoning model, similar to [Llama Nemotron reasoning models](https://www.nvidia.com/en-us/ai-data-science/foundation-models/llama-nemotron/).
2. Apply Fine Tuning with NeMo 2.0, using LoRA adapters or full model fine-tuning.
3. Train using NeMo's distributed, mixed-precision trainer.
4. Save a fine-tuned checkpoint ready for evaluation or deployment.


### 🧰 Tools and Resources
* [NeMo Framework](https://docs.nvidia.com/nemo-framework/user-guide/latest/overview.html)
* [Llama-Nemotron-Post-Training-Data](https://huggingface.co/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset), an open source dataset for instilling reasoning behavior in large language models.
* [NeMo Curator](https://github.com/NVIDIA/NeMo-Curator) for data curation

## 📌 Requirements

### Prerequisites

* Access to latest NeMo Framework NGC Containers
* This playbook has been tested on `nvcr.io/nvidia/nemo:25.04.01`. It is expected to work similarly on other environments.
* A valid Hugging Face API token with access to the [Meta LLaMa 3.1-8B Instruct](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct) model (since this is a gated model).

### Dataset
To follow along, you would need an appropriate reasoning dataset. Checkout the tutorial on [curating the Llama Nemotron Reasoning Dataset with NVIDIA NeMo Curator](https://github.com/NVIDIA/NeMo-Curator/tree/main/tutorials/llama-nemotron-data-curation).
You will need the output from that tutorial as the training set input to this playbook!

### Hardware Requirements

You can either fine-tune a LoRA adapter, or fine-tune the entire model.

* **Parameter Efficient Fine-Tuning via LoRA adapters**: 
  * This playbook has been verified on **a single H100 80GB** when PEFT is enabled.
  * It takes **48 hours** to train the model for 2000 steps and observe reasoning behavior.
* **Full Fine-Tuning**:
  * This playbook has been tested on **8xH100 80GB** GPUs. You can scale training to more GPUs as well as multiple nodes by modifying the appropriate parameters.
  * It takes **12 hours** to train the model for 2000 steps and observe reasoning behavior.


Let's dive in!

## 🚀 Step 0. Launch the NeMo Framework container

Run the following command to launch the NeMo Framework training container. Ensure to populate the `HF_TOKEN` variable with a valid API key:

```bash
docker run -it --rm \
    --gpus all --shm-size=16GB \
    --ipc=host --network host \
    -v $(pwd):/workspace \
    -e HF_TOKEN=<Your Hugging Face API token> \
    nvcr.io/nvidia/nemo:25.04.01
```

#### Launch Jupyter Notebook as follows:

```bash
jupyter notebook --allow-root --ip 0.0.0.0 --port 8088 --no-browser --NotebookApp.token=''
```

## 📋 Step 1. Preparations
Specify the parameters you will use for model training. These include:

1. Whether you will do PEFT/LoRA or full fine-tuning.
2. How many GPUs you will use for training.
3. How many steps you will do training for. 
    - ⚠️ **At least 2000 steps is needed to observe reasoning behavior.**
4. The location where the training dataset is stored. This dataset is expected to be in the structured JSONL format, stored locally with the name `training.jsonl`.
5. Where to store the Hugging Face model checkpoint that is converted to NeMo format.
6. The location where the training outputs will be stored.

Setup the values in the cell below, then execute it.

In [ ]:
import os

# LoRA or full model training? Set to `False` for full model training.
DO_LORA_ADAPTER_TRAINING = True
# The number of GPUs used for training
N_DEVICES = 1
# The number of training steps (at least 2000 steps are required for a good reasoning model)
N_STEPS = 2000
# Where the training data is stored
TRAINING_DATASET_DIR = "/path/to/your/data_directory"
# Where the NeMo conversions are stored
NEMO_CONVERSION_DIR = "/path/to/your/nemo_conversions"
# Where the checkpoints and other experiment results are stored
OUTPUT_DIR = "/path/to/your/output_directory"

#
# Some basic sanity checks
#
assert os.path.exists(TRAINING_DATASET_DIR), "Data root directory does not exist. Please set DATA_ROOT to a valid path."
# Ensure training.jsonl exists in the data directory
assert os.path.exists(os.path.join(TRAINING_DATASET_DIR, "training.jsonl")), "training.jsonl file does not exist in the specified DATA_ROOT."
# Ensure the NeMo converstion directory exists
assert os.path.isdir(NEMO_CONVERSION_DIR), "NeMo conversion directory does not exist. Please set NEMO_CONVERSION_DIR to a valid path."
assert os.path.isdir(OUTPUT_DIR), "Output directory does not exist. Please set OUTPUT_DIR to a valid path."



### Convert HuggingFace Checkpoint to NeMo Format

Before training, we need to convert the HuggingFace LLaMA 3.1–8B Instruct checkpoint into NeMo format. NeMo provides a built-in utility ```llm.import_ckpt()``` to handle this conversion.
After conversion, the model can be loaded and fine-tuned using NeMo APIs directly.

⚠️ This step only needs to be run once per model.

In [ ]:
import nemo_run as run
from nemo import lightning as nl
from nemo.collections import llm

import pytorch_lightning as pl
from nemo.collections.llm.recipes.precision.mixed_precision import bf16_mixed
from datetime import datetime

# Configure the import from HuggingFace format to NeMo format
def configure_checkpoint_conversion():
    return run.Partial(
        llm.import_ckpt,
        model=llm.llama31_8b.model(),  # Predefined LLaMA 3.1 8B model structure
        source="hf://meta-llama/Llama-3.1-8B-Instruct",  # Path to HF checkpoint (local or HF hub)
        output_path=NEMO_CONVERSION_DIR,  # Directory to save the converted NeMo checkpoint
        overwrite=True,  # Setting this to False will result in an error if the checkpoint was already converted
    )

# Create the configured import task
import_ckpt = configure_checkpoint_conversion()

# Define the local executor (single-node)
local_executor = run.LocalExecutor()

# Execute the checkpoint conversion
run.run(import_ckpt, executor=local_executor)

✓ Checkpoint imported to /root/.cache/nemo/models/Meta-Llama-3.1-8B

## 📂 Step 2. Prepare Data

Next, define the configuration for loading and preprocessing an instruction-tuning dataset using NeMo’s FineTuningDataModule. The dataset is expected to be in a structured format (e.g. JSONL), stored locally as ```training.jsonl```.

The training-related parameters like batch size, number of workers, memory mapping, and device count can be modified based on the size of the model, dataset size and compute resources available. We recommend a context length of at least 8192 for best results.

⚠️ For this particular dataset, the global batch size must be at least 256 for best results. You can increase this value based on compute resources available to you.

⚠️ If you observe any out-of-memory issues with CPU memory (i.e. RAM), you can decrease `num_workers`.

#### A Note on Batching and Gradient Accumulation
The parameter `micro_batch_size` specified how many batches of data are assigned to **each** GPU. The `global_batch_size` parameter specifies the total batch size across **all** devices, **including gradient accumulation**.

Note that the computation for the amount of gradient accumulation required is done internally by NeMo, and is based on how many GPUs are used for training. In the example below, if we are using a single GPU with `micro_batch_size=1` and `global_batch_size=256`, the number of gradient accumulation steps will be set to `256`.

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d-%H%M")
experiment_name = "baseline-lora-only" if DO_LORA_ADAPTER_TRAINING else "baseline-full-model"

# Define fine-tuning dataset configuration
finetune_config = run.Config(
    llm.FineTuningDataModule,
    dataset_root=TRAINING_DATASET_DIR,  # Path to your preprocessed dataset (JSONL, etc.)
    seq_length=8192,                    # Max sequence length for input tokens
    micro_batch_size=1,                 # Per-device batch size. Each GPU will process this many samples at a time
    global_batch_size=256,              # Total batch size across all devices
    seed=1234,                          # Seed for reproducibility
    memmap_workers=1,                   # Use memory-mapped dataset format for performance
    num_workers=8,                      # DataLoader worker threads
    pin_memory=True,                    # Optimize data transfer to GPU
)

## 🛠️ Step 3. Configure Fine Tuning with the NeMo 2.0 API

In this step, we'll use the modular NeMo 2.0 API to configure:

* The distributed trainer

* Logging and checkpointing

* Optimizer with cosine annealing scheduler

* Model definition and resume behavior

* Final recipe assembly for fine-tuning

### ⚙️ 3.1 Configure the Trainer

We define the training strategy with Megatron's Distributed Training strategy using tensor model parallelism and enabling mixed precision with bf16.

For this excercise, we do not have a validation set, so all the settings for validation are set to 0. If you decide to use a validation set, you would need to update these parameters.

In [ ]:
def trainer() -> run.Config[nl.Trainer]:
    strategy = run.Config(
        nl.MegatronStrategy,
        tensor_model_parallel_size=1,
        optimizer_cpu_offload=True
    )
    trainer = run.Config(
        nl.Trainer,
        devices=N_DEVICES,
        num_nodes=1,             # Change to >1 for multi-node training
        max_steps=N_STEPS,
        accelerator="gpu",
        strategy=strategy,
        plugins=bf16_mixed(),
        log_every_n_steps=10,
        limit_val_batches=0,
        val_check_interval=0,
        num_sanity_val_steps=0,
        use_distributed_sampler=False,
    )
    return trainer

### 📝 3.2 Configure Logging and Checkpointing
This cell defines the logging mechanism and model checkpointing during training.

In [ ]:
def logger() -> run.Config[nl.NeMoLogger]:
    ckpt = run.Config(
        nl.ModelCheckpoint,
        save_last=True,
        every_n_train_steps=100,  # Checkpoint frequency
        monitor="reduced_train_loss",
        save_top_k=2,  # How many top checkpoints to keep
        save_on_train_epoch_end=True,
        save_optim_on_train_end=True,
    )

    return run.Config(
        nl.NeMoLogger,
        name=f"{experiment_name}-trained-model-checkpoints",
        log_dir=f"{OUTPUT_DIR}/results-{timestamp}-{N_DEVICES}-devices-{experiment_name}",
        use_datetime_version=True,
        ckpt=ckpt,
        wandb=None
    )

### 📈 3.3 Configure Optimizer with Cosine Annealing
We use the Adam optimizer with gradient clipping, distributed optimizer support, and a cosine annealing learning rate schedule.

We recommend starting off with a high learning rate (`1e-4`) and gradually decreasing it for best results. For this recipe, we use a 5% warmup schedule. We recommend setting this to be between 0% to 10% of the training for most use-cases.

In [ ]:
from nemo.lightning.pytorch.optim import CosineAnnealingScheduler
from megatron.core.optimizer import OptimizerConfig

def lr_scheduler():
    warmup_steps = int(N_STEPS * 0.05)  # 5% of total steps for warmup
    return run.Config(
        CosineAnnealingScheduler,
        warmup_steps=warmup_steps,
        constant_steps=0,  # Do not fix the learning rate, always decrease it
        min_lr=1e-5,
    )

def adam_with_cosine_annealing() -> run.Config[nl.OptimizerModule]:
    opt_cfg = run.Config(
        OptimizerConfig,
        optimizer="adam",
        lr=1e-4,
        weight_decay=0.001,
        use_distributed_optimizer=True,
        clip_grad=1.0,
        bf16=True,
    )

    return run.Config(
        nl.MegatronOptimizerModule,
        config=opt_cfg,
        lr_scheduler=lr_scheduler(),
    )

### 🧠 3.4 Define the Base Model and Resume Logic
We use the built-in LLaMA 3.1 8B config from NeMo and optionally resume from a previously saved checkpoint.

In [ ]:
def llama31_8b() -> run.Config[pl.LightningModule]:
    return run.Config(llm.LlamaModel, config=run.Config(llm.Llama31Config8B))

def resume() -> run.Config[nl.AutoResume]:
    return run.Config(
        nl.AutoResume,
        restore_config=run.Config(
            nl.RestoreConfig,
            path=NEMO_CONVERSION_DIR,
        ),
        resume_if_exists=True,
    )

### 📦 3.5 Assemble the Fine-Tuning Recipe
This ties together the model, trainer, dataset config, optimizer, and logger into a single training recipe using NeMo’s run.Partial system.

#### Recommended LoRA settings
We recommend using a LoRA rank of at least 64 with `alpha = 128` for good results, as lower ranks did not perform as well in our experiments.

#### Preventing Out-of-Memory Errors for Full Fine-
Fine-tuning an entire 8-billion parameter model requires a lot of GPU memory, and even 8xH100 80GB might not be enough, depending on the model's context size. One way to save memory is to enable [activation recomputation](https://docs.nvidia.com/nemo-framework/user-guide/latest/nemotoolkit/features/optimizations/activation_recomputation.html). This approach significantly reduces the memory requirements at the cost of slower training speeds.

In the cell below, we use the most aggressive form of activation recomputation to ensure this notebook can be executed on a single node with 8x80GB GPUs. For more information on available options please refer to the [documentation](https://docs.nvidia.com/nemo-framework/user-guide/latest/nemotoolkit/features/optimizations/activation_recomputation.html).

In [ ]:
def configure_finetuning_recipe():
    peft = None

    # Are we training a LoRA adapter or the full model?
    if DO_LORA_ADAPTER_TRAINING:
        peft = llm.peft.LoRA(
            dim=64,
            alpha=128,
        )

    recipe = run.Partial(
        llm.finetune,
        model=llama31_8b(),
        trainer=trainer(),
        data=finetune_config,  # From earlier step
        log=logger(),
        optim=adam_with_cosine_annealing(),
        resume=resume(),
        peft=peft,  # LoRA adapter configuration if applicable
    )

    if not DO_LORA_ADAPTER_TRAINING:
        # Use aggressive recomputation settings for full model training to save on GPU memory
        recipe.model.config.recompute_granularity = "full"
        recipe.model.config.recompute_method = "uniform"
        recipe.model.config.recompute_num_layers = 1

    return recipe

## ▶️ Step 4: Run Fine-Tuning with NeMo 2.0 and nemo-run
Now that everything is configured (model, trainer, optimizer, logging, and data), it's time to launch the training job using nemo-run's LocalExecutor.

This will:

* Use torchrun to launch a multi-GPU job
* Set environment variables for optimized NCCL behavior
* Kick off the training loop with your full configuration

While the training is ongoing, you can monitor the progress via tensorboard.

In [ ]:
def local_executor_torchrun() -> run.LocalExecutor:
    # Environment variables to optimize distributed training
    env_vars = {
        "TORCH_NCCL_AVOID_RECORD_STREAMS": "1",
        "NCCL_NVLS_ENABLE": "0",
    }

    return run.LocalExecutor(
        ntasks_per_node=N_DEVICES,
        launcher="torchrun",
        env_vars=env_vars,
    )

# Execute the training run
if __name__ == '__main__':
    recipe = configure_finetuning_recipe()
    print(recipe)
    run.run(
        recipe,
        executor=local_executor_torchrun()
    )

For your reference here are the loss plots from our own experiments using 500,000 training samples, with a batch size of 256 and 2000 training steps.

You might be wondering about the sudden loss drop at the end. This is expected!
The training dataset is arranged in the increasing order of sample difficulty (i.e. curriculum learning).
With 500,000 training samples, a batch size of 256 and 2000 steps, that’s just slightly over 1 epoch of training.
Towards the end of that epoch, when the model sees the first few (easier samples) again, it can easily predict the right tokens for them so the loss ends up being much lower.

#### LoRA Training Loss Plots
![LoRA Training Loss Plots](images/loss-plot-lora.png)

#### Full Fine-tuning Loss Plots
![Fine-tuning Loss Plots](images/loss-plot-full-finetuning.png)


## 🎉 Tada! You Just Trained Your First Reasoning Model!
Congratulations — you've successfully fine-tuned LLaMA 3.1-8B Instruct into a reasoning model using NeMo Framework!

Your model is now ready to:

* Solve problems through reasoning and analysis.
* Selectively perform reasoning (i.e. `reasoning on` or `reasoning off`).

### 🚀 Next Steps
* 🧪 Evaluate your model on reasoning benchmarks (e.g., MMLU, GSM8K)
* ☁️ Package the model for deployment or inference with Triton or vLLM
* 📤 Optionally, upload it to HuggingFace or NGC to share with the world